In [25]:
!pip install -U langchain
!pip install -U langchain-core
!pip install -U langchain-community
!pip install -U wikipedia
!pip install -U langchain-groq
!pip install -U pydantic==2.7.4

  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached pydantic_core-2.18.4-cp312-none-win_amd64.whl.metadata (6.7 kB)
Using cached pydantic-2.7.4-py3-none-any.whl (409 kB)
Using cached pydantic_core-2.18.4-cp312-none-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.22 requires langchain-core<0.3.0,>=0.2.33, but you have langchain-core 0.3.10 which is incompatible.
langchain-openai 0.1.22 requires openai<2.0.0,>=1.40.0, but you have openai 0.28.0 which is incompatible.
langserve 0.2.3 requires langchain-core<0.3,>=0.1, but you have langchain-core 0.3.10 which is incompatible.


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model="llama3.2:1b"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F32641DD30>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retriever_tool.name

'langsmith_search'

In [9]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [10]:
tools=[wikipedia,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\nikhi\\Desktop\\Projects\\LLM activity\\LLM test 1\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F3777EB740>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings']

In [11]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192", temperature=0).bind_tools(tools)

In [15]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [17]:
agent_executor.invoke({"input":"Tell me about Langsmith"})

{'input': 'Tell me about Langsmith',
 'output': 'It looks like the tool call yielded a lot of information about LangSmith! Based on the output, it seems that LangSmith is a platform for building and evaluating AI models, specifically designed for LangChain users. The output provides a getting started guide, highlighting the different ways to log traces to LangSmith, as well as a sample dataset and an example of how to use the LangSmith SDK.\n\nIt appears that LangSmith provides a range of tools and features for building and evaluating AI models, including data creation, evaluation, and logging. The output also mentions the LangChain Python and JS/TS SDKs, which can be used to integrate LangSmith into applications.\n\nOverall, it seems that LangSmith is a powerful platform for building and evaluating AI models, and the output provides a good starting point for learning more about it.'}

In [18]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, published in 2016, studies graphs on lattice points with edges coming from a finite set of allowed moves.'}